In [2]:
import spacy
from spacy import displacy
import spacy_annotator as spa
import pandas as pd
import numpy as np
import os

os.getcwd()

'c:\\Users\\izard\\Documents\\GitHub\\spacy-ner-finetuned-english'

In [2]:
! pip install spacy_annotator

  Using cached spacy_annotator-2.1.3-py3-none-any.whl (6.0 kB)
  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
platformdirs 3.11.0 requires typing-extensions>=4.7.1; python_version < "3.8", but you have typing-extensions 4.4.0 which is incompatible.
flake8 5.0.4 requires importlib-metadata<4.3,>=1.1.0; python_version < "3.8", but you have importlib-metadata 6.7.0 which is incompatible.
fake-useragent 1.2.1 requires importlib-metadata~=4.0; python_version < "3.8", but you have importlib-metadata 6.7.0 which is incompatible.


In [3]:
#nlp = spacy.blank("en") # load a new spacy model for first time 
nlp = spacy.load("output/model-best") # load custom model after first time train

c:\Users\izard\miniconda3\envs\venv\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.5.2 and may not be 100% compatible with the current version (3.8.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [4]:
# prepare train data
import spacy_annotator as spa

train_data=pd.DataFrame({
    "text":pd.read_parquet('./sample.parquet').loc[91:100]['Content'].tolist()})

train_data=train_data[~((train_data['text'].str.contains('[\u4e00-\u9fff]+'))|(train_data['text'].isnull()))].reset_index(drop=True)
train_data['text']=train_data['text'].str.replace(r'(.+\d\)\:)|(.+[A-Z]\):)|(.+[A-Z]:)','',regex=True).str.strip()
train_data

,text
0,S P Setia is expanding its footprint in Austra...
1,"Domestic flights were cancelled and almost 3,0..."
2,RAM Ratings has placed KLK and Batu Kawan on a...
3,"+4.8%), reaching RM587.5b contribution to tota..."
4,Prime Minister Datuk Seri Anwar Ibrahim will l...
5,Hong Leong Financial Group’s FY23 Core net pro...
6,"Bank Negara Malaysia had on 18 May 2023, impos..."
7,"Bank Negara Malaysia had, on 29 December 2022,..."
8,Indonesia’s Nusantara Capital City Authority i...
9,ASEANmember countries need to collaborate to m...


### Reference for Text Annotation

> - <sup>PERSON:      People, including fictional.</sup>
> - <sup>NORP:        Nationalities or religious or political groups.</sup>
> - <sup>FAC:         Buildings, airports, highways, bridges, etc.</sup>
> - <sup>ORG:         Companies, agencies, institutions, etc.</sup>
> - <sup>GPE:         Countries, cities, states.</sup>
> - <sup>LOC:         Non-GPE locations, mountain ranges, bodies of water.</sup>
> - <sup>PRODUCT:     Objects, vehicles, foods, etc. (Not services.)</sup>
> - <sup>EVENT:       Named hurricanes, battles, wars, sports events, etc.</sup>
> - <sup>WORK_OF_ART: Titles of books, songs, etc.</sup>
> - <sup>LAW:         Named documents made into laws.</sup>
> - <sup>LANGUAGE:    Any named language.</sup>
> - <sup>DATE:        Absolute or relative dates or periods.</sup>
> - <sup>TIME:        Times smaller than a day.</sup>
> - <sup>PERCENT:     Percentage, including ”%“.</sup>
> - <sup>MONEY:       Monetary values, including unit.</sup>
> - <sup>QUANTITY:    Measurements, as of weight or distance.</sup>
> - <sup>ORDINAL:     “first”, “second”, etc.</sup>
> - <sup>CARDINAL:    Numerals that do not fall under another type.</sup>

In [6]:
annotator=spa.Annotator(labels=["GPE","PERSON","ORG","FAC","MONEY","NORP","LOC","PRODUCT","EVENT","PERCENT","WORK_OF_ART","TIME","ORDINAL","CARDINAL","QUANTITY","LAW"],model=nlp) #annotate train data

In [7]:
df_labels=annotator.annotate(df=train_data,col_text="text",shuffle=False)

HTML(value='-1 examples annotated, 11 examples left')

Text(value='', description='GPE', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

Text(value='', description='PERSON', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

Text(value='', description='ORG', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

Text(value='', description='FAC', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

...

Text(value='', description='ORDINAL', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

Text(value='', description='CARDINAL', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

Text(value='', description='QUANTITY', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

Text(value='', description='LAW', layout=Layout(width='auto'), placeholder='ent one, ent two, ent three')

Output()

In [ ]:
spacy_annotations = annotator.to_spacy(df_labels,"./train/train_1_100.spacy")

In [5]:
# to re-wrote config.cfg file
! python -m spacy init fill-config base_config_gpu.cfg config.cfg


✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


c:\Users\izard\miniconda3\envs\venv\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\izard\miniconda3\envs\venv\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [1]:
# to train
! python -m spacy train config.cfg --output ./output --paths.train ./train --paths.dev ./train --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ------  ------  ------  ------
  0       0         900.59   1836.22    0.00    0.00    0.01    0.00
  0     200      249763.63  69446.24   45.53   59.09   37.03    0.46
  0     400       47992.26  14496.94   70.86   67.69   74.34    0.71
  1     600       20601.92   9073.42   67.50   81.43   57.63    0.67
  1     800        6937.75   8577.04   76.42   72.35   80.97    0.76
  2    1000        4333.47   7674.95   80.23   75.58   85.50    0.80
  2    1200        4707.43   7126.67   80.93   74.97   87.94    0.81
  2    1400        3319.39   6469.11   82.31   80.99   83.68    0.82
  3    1600     

c:\Users\izard\miniconda3\envs\venv\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\izard\miniconda3\envs\venv\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\izard\miniconda3\envs\venv\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\izard\miniconda3\envs\venv\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please us